In [60]:
import hvplot.pandas
import pandas as pd
import requests
import json
from api_keys import geoapify_key

In [92]:
philly_crime_url = 'https://phl.carto.com/api/v2/sql?q=SELECT * FROM arrests_zipcode'
philly_crime = requests.get(philly_crime_url).json()
philly_crime = pd.DataFrame(philly_crime['rows'])
philly_crime = philly_crime.loc[philly_crime['zip_code'] != 'Unknown',:]
philly_crime = philly_crime.sample(5)
philly_crime

,cartodb_id,the_geom,the_geom_webmercator,offense_category,day,zip_code,defendant_race,count,objectid
53998,53999,None,None,Criminal Mischief,2022-10-31T04:00:00Z,19143,Black,1,26295532
364747,364748,None,None,Victim/Witness Intimidation & Retaliation,2011-02-15T05:00:00Z,19136,White,1,26597351
357136,357137,None,None,Uncategorized Offenses,2017-01-20T05:00:00Z,19133,Latinx,1,26574007
153452,153453,None,None,DUI,2012-08-04T04:00:00Z,19120,Black,1,26386950
337148,337149,None,None,Uncategorized Offenses,2014-11-11T05:00:00Z,19123,Black,1,26563016


In [99]:
philly_crime['lat'] = ''
philly_crime['lon'] = ''
for index, row in philly_crime.iterrows():
    zipcode = row['zip_code']
    zipcodeurl = f'https://api.geoapify.com/v1/geocode/search?text={zipcode}&lang=en&limit=10&type=postcode&filter=countrycode:us&apiKey={geoapify_key}'
    zipcodecoords = requests.get(zipcodeurl).json()
    print(f'Converting zip code {zipcode} to coordinates')
    try:
        lat = zipcodecoords['features'][0]['properties']['lat']
        lon = zipcodecoords['features'][0]['properties']['lon']
        philly_crime['lat'] = lat
        philly_crime['lon'] = lon
        print(lat,lon)
    except:
        lat = ''
        lon = ''
        print('Bad item')
        pass

Converting zip code 19143 to coordinates
39.943270181 -75.222942213
Converting zip code 19136 to coordinates
40.041977751 -75.02558942
Converting zip code 19133 to coordinates
39.988751517 -75.1420438
Converting zip code 19120 to coordinates
40.028970779 -75.121499097
Converting zip code 19123 to coordinates
39.963974951 -75.14735507


In [103]:
%%capture --no-display
themap = philly_crime.hvplot.points("lon","lat",geo = True,tiles = "OSM",frame_width = 800,frame_height = 600)

In [101]:
philly_crime

,cartodb_id,the_geom,the_geom_webmercator,offense_category,day,zip_code,defendant_race,count,objectid,lat,lon
53998,53999,None,None,Criminal Mischief,2022-10-31T04:00:00Z,19143,Black,1,26295532,39.963975,-75.147355
364747,364748,None,None,Victim/Witness Intimidation & Retaliation,2011-02-15T05:00:00Z,19136,White,1,26597351,39.963975,-75.147355
357136,357137,None,None,Uncategorized Offenses,2017-01-20T05:00:00Z,19133,Latinx,1,26574007,39.963975,-75.147355
153452,153453,None,None,DUI,2012-08-04T04:00:00Z,19120,Black,1,26386950,39.963975,-75.147355
337148,337149,None,None,Uncategorized Offenses,2014-11-11T05:00:00Z,19123,Black,1,26563016,39.963975,-75.147355


In [104]:
themap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]